In [5]:
from os.path import exists
import requests
import pandas as pd
import numpy as np
from collections import defaultdict
import ast
from json import dump, load
from IPython.display import clear_output

In [6]:
START = 0
DAILY_MAX = 2000
CBD_COORDS = [144.946457,-37.840935]

In [3]:
# Read in API keys
with open('../data/raw/api.json', 'r') as f:
  keys = load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/api.json'

In [4]:
key_vals = list(keys.values())
num_keys = len(key_vals)

NameError: name 'keys' is not defined

In [7]:
# Read in coordinate data
data = pd.read_csv('../data/curated/bbox.csv', index_col=0)
end = len(data)

In [9]:
# Check if .json file already exists, and create file if not
if not exists('../data/raw/distance.json'):
    with open('../data/raw/distance.json', 'w') as f:
        dump([], f)

In [24]:
for i in range(START, end):
    clear_output(wait=True)
    print(f'Beginning query number {i}:')
    
    
    data_dict = defaultdict(dict)
    body = {"coordinates":[[data.long[i], data.lat[i]], CBD_COORDS]}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': f'{key_vals[i%num_keys]}',
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)

    print(f'Code {call.status_code}, {call.reason}')
    
    # Index through dictionary to obtain total diatcnce (in metres) 
    # and trip duration (in seconds) to reach Melbourne CBD
    try:
        data_dict[data.coordinates[i]] = ast.literal_eval(call.__dict__['_content'] \
                                                        .decode('utf-8'))['routes'][0]['summary']
    except:
        data_dict[data.coordinates[i]] = dict({"distance": np.nan, "duration": np.nan})
    
    
    with open('../data/raw/distance.json', 'r') as f:
        file = load(f)
    file.append(data_dict)
    with open('../data/raw/distance.json', 'w') as f:
        dump(file, f, indent=4, separators=(',',': '))

Beginning query number 6599:
Code 200, OK
